# Preparation

In [ ]:
import torch
torch.__version__

In [ ]:
import os
import pandas as pd
import numpy as np
import random
from torch import nn

from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

from model import *
from dataset import *

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
BATCH_SIZE = 256
NUMEPOCHS = 50

# Load data & data preprocess

In [ ]:
# import pandas as pd
# df = pd.read_csv("Data/ml-1m.train.rating",sep='\t', header=None, names=['uid', 'mid', 'rating', 'timestamp'], engine='python')
# df

In [ ]:
# df2 = pd.read_csv("Data/ml-1m.test.negative",sep = "\t",header=None)
# df2

# Trian and Test

### Compare the three methods 

In [ ]:
config = {
    'num_users': num_of_user,
    'num_items': num_of_item,
    'latent_dim': 8,
    "layers_num(X)" : 4,
    'layers': [64, 32, 16, 8],
    'model_type': 'GMF'     #　MLP, NeuMF
}

## Reproduce the ablation study --- MLP with different layers

 K: 8, 16, 32, 64
 
 layers (X): 0, 1, 2, 3, 4